In [2]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import yaml
from keras import (
    Sequential,
    layers,
    optimizers,
    losses,
    metrics,
    callbacks,
    regularizers,
)
from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [3]:
class DataTransformation:
    def __init__(self, config: dict) -> None:
        self.image_size = (
            config["image_height"],
            config["image_width"],
        )
        self.image_channels_expactation = config["image_channels_expectation"]
        self.class_names = []
        self.batch_size = config["batch_size"]

    def get_dataset(self, path: Path) -> tuple[tf.data.Dataset]:

        print(f"Loading dataset from: {path}")
        train_ds = tf.keras.utils.image_dataset_from_directory(
            path / "train",
            seed=242,
            image_size=self.image_size,
            batch_size=self.batch_size,
            shuffle=True,
            label_mode="categorical",
            color_mode="grayscale" if self.image_channels_expactation == 1 else "rgb",
            interpolation="gaussian",
            validation_split=0.1,
            subset="training",
        )
        val_ds = tf.keras.utils.image_dataset_from_directory(
            path / "test",
            seed=242,
            image_size=self.image_size,
            batch_size=self.batch_size,
            shuffle=True,
            label_mode="categorical",
            color_mode="grayscale" if self.image_channels_expactation == 1 else "rgb",
            interpolation="gaussian",
            validation_split=0.1,
            subset="validation",
        )
        self.class_name = train_ds.class_names

        for image, label in train_ds:
            print(f"Batched image shape: {image.shape}")
            print(f"Batched label shape: {label.shape}")
            break

        AUTOTUNE = tf.data.AUTOTUNE
        train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
        val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
        return train_ds, val_ds

In [ ]:
params = {
    "CNN": {
        "input_shape": [32, 32, 1],
        "num_classes": 6,
        "conv_units": [32, 64, 128, 512, 512],
        "dense_units": [256, 512],
        "dropout_rate": 0.25,
        "learning_rate": 0.001,
        "l1": 0.004,
        "l2": 0.004,
    },
    "MLP": {
        "input_shape": [32, 32, 1],
        "num_classes": 6,
        "dense_units": [128, 32],
        "dropout_rate": 0.5,
        "learning_rate": 0.001,
        "l1": 0.004,
        "l2": 0.004,
    },
    "ViT": {
        "input_shape": [32, 32, 1],
        "num_classes": 6,
        "dense_units": [6],
        "dropout_rate": 0.5,
        "learning_rate": 0.001,
        "l1": 0.004,
        "l2": 0.004,
    },
    "early_stopping": {"monitor": "val_loss", "patience": 10},
    "reduce_lr": {
        "monitor": "val_loss",
        "patience": 10,
        "factor": 0.1,
        "min_lr": 1e-06,
    },
}

data_augmentation = Sequential(
    [
        layers.Rescaling(1.0 / 255),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)


early_stop = callbacks.EarlyStopping(
    monitor=params["early_stopping"]["monitor"],
    patience=params["early_stopping"]["patience"],
    verbose=0,
    mode="auto",
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor=params["reduce_lr"]["monitor"],
    factor=params["reduce_lr"]["factor"],
    patience=params["reduce_lr"]["patience"],
    min_lr=params["reduce_lr"]["min_lr"],
    mode="auto",
    verbose=0,
)


def evaluate_model(model: keras.Sequential, test_ds: tf.data.Dataset):
    result = model.evaluate(test_ds)
    print(f"Evaluating {model._name} model: {result}")

    y_pred = model.predict(test_ds)
    y_pred = y_pred.argmax(axis=1)

    y_true = test_ds.map(lambda x, y: y)
    ConfusionMatrixDisplay(confusion_matrix(y_true, y_pred)).plot()
    os.makedirs(name=f"visualize/{model._name}", exist_ok=True)
    plt.savefig(f"visualize/{model._name}/confusion_matrix.png")
    return result


def fit_model(
    model: keras.Sequential,
    train_ds: tf.data.Dataset,
    val_ds: tf.data.Dataset,
    epochs: int = 10,
) -> callbacks.History:
    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=val_ds,
        callbacks=[
            early_stop,
            reduce_lr,
            callbacks.ModelCheckpoint(
                filepath=f"weights/{model._name}/best.keras",
                monitor="val_f1_score",
                save_best_only=True,
                verbose=0,
            ),
            callbacks.ModelCheckpoint(
                filepath=f"weights/{model._name}/last.keras",
                monitor="val_loss",
                verbose=0,
            ),
            callbacks.CSVLogger("logs/" + model._name + ".log"),
        ],
    )
    return history


class CNN:
    def __init__(self, params) -> None:
        self.input_shape = params["input_shape"]
        self.num_classes = params["num_classes"]
        self.conv_units = params["conv_units"]
        self.dense_units = params["dense_units"]
        self.dropout_rate = params["dropout_rate"]
        self.initial_lr = params["learning_rate"]
        self.l1 = params["l1"]
        self.l2 = params["l2"]

    @property
    def build_model(self) -> Sequential:
        model = Sequential(
            [layers.Input(self.input_shape), data_augmentation], name="CNN"
        )

        for units in self.conv_units:
            model.add(layers.Conv2D(units, 3, activation="relu", padding="same"))
            model.add(layers.BatchNormalization())
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))
            model.add(layers.Dropout(self.dropout_rate))

        model.add(layers.Flatten())
        for units in self.dense_units:
            model.add(layers.Dense(units, activation="relu"))
            model.add(layers.Dropout(self.dropout_rate))

        model.add(
            layers.Dense(
                self.num_classes,
                activation="softmax",
                name="output",
            ),
        )
        model.compile(
            optimizer=optimizers.Adam(learning_rate=self.initial_lr),
            loss=losses.CategoricalCrossentropy(),
            metrics=[
                metrics.CategoricalAccuracy(),
                metrics.AUC(),
                metrics.Precision(),
                metrics.Recall(),
                metrics.F1Score(average="weighted"),
            ],
        )
        model._name = "CNN"
        print(f"{model._name} model summary: ")
        print(model.summary())
        os.makedirs(name=f"visualize/{model._name}", exist_ok=True)
        keras.utils.plot_model(
            model,
            to_file=f"visualize/{model._name}/layers_structure.png",
            show_shapes=True,
            show_layer_names=True,
        )
        return model

In [ ]:
config = {
    "data_ingestion": {
        "data_path": "dataset/",
    },
    "data_transformation": {
        "image_width": 32,
        "image_height": 32,
        "image_channels_expectation": 1,
        "batch_size": 32,
    },
}

In [ ]:
# Get dataset
data_transformation = DataTransformation(config=config["data_transformation"])
train_ds, val_ds = data_transformation.get_dataset(
    path=Path(config["data_ingestion"]["data_path"]),
)

# Train model
cnn = CNN(params=params["CNN"]).build_model
history = fit_model(model=cnn, train_ds=train_ds, val_ds=val_ds, epochs=10)

# Show result of training
for metric in history.history.keys():
    if "val" in metric:
        plt.scatter(
            history.epoch,
            history.history[metric],
            label=metric,
            color="orange",
        )
    else:
        plt.plot(
            history.epoch,
            history.history[metric],
            label=metric,
            color="blue",
        )
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.title(f"{cnn._name}: {metric}")
    os.makedirs(name=f"visualize/{cnn._name}", exist_ok=True)
    plt.savefig(f"visualize/{cnn._name}/{metric}.png")